<a href="https://colab.research.google.com/github/vvikasreddy/JargonAI/blob/main/Summa_rizzer_on_custom_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing and Downloading libraries

In [1]:
# downloading necessary libraries
!pip install datasets
!pip install rouge_score
!pip install bert-score
!pip install evaluate

In [2]:
# importing libraries
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AdamW
from torch.utils.data import DataLoader, Dataset
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from sklearn.model_selection import train_test_split
import textwrap
from tqdm import tqdm

# Loading the data and Pre-processing




In [3]:
#loading the dataset
dataset_path = "/content/Corpus_all.csv"
data = pd.read_csv(dataset_path, encoding="ISO-8859-1")

In [4]:
data.head()

,Document,Summary
0,412.106.1 English is not an official language ...,The Swiss Federal University for Vocational Ed...
1,944.021.1 English is not an official language ...,The EAER Ordinance on the Declaration for Timb...
2,Title and commencement 1 This order may be cit...,The North West Water Authority (Solway Firth) ...
3,Citation and commencement 1 This order may be ...,The Trafford Park Development Corporation (Are...
4,"Title, commencement and interpretation 1 1 Thi...",The North West Water Authority (Returns of Eel...


In [5]:

# putting the text and summaries into the containers
input_texts = data["Document"].tolist()
target_texts = data["Summary"].tolist()

In [6]:
# taking a look at the first index
print(input_texts[0])
print(target_texts[0])

412.106.1 English is not an official language of the Swiss Confederation. This translation is provided for information purposes only, has no legal force and may not be relied on in legal proceedings. Ordinance on the Swiss Federal University for Vocational Education and Training(SFUVET Ordinance)of 18 June 2021 (Status as of 1 August 2021)The Swiss Federal Council,on the basis of Article 35 of the SFUVET Act of 25 September 20201,ordains:1 SR 412.106Art. 1 Registered location The Swiss Federal University for Vocational Education and Training (SFUVET) shall be based in Zollikofen.Art. 2 Regional campuses SFUVET shall offer its services through three regional campuses: one in the German-speaking region, one in the French-speaking region and one in the Italian-speaking region of Switzerland.Art. 3 Federal Council's strategic objectives The Federal Department of Economic Affairs, Education and Research (EAER) shall submit SFUVET's strategic objectives drafted by the Federal Council to the 

In [7]:
#initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
#define the custom dataset class

class CustomDataset(Dataset):
    def __init__(self, tokenizer, input_texts, target_texts, max_input_length=512, max_target_length=128):
        self.tokenizer = tokenizer
        self.input_texts = input_texts
        self.target_texts = target_texts
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, idx):
        input_encoding = self.tokenizer(
            self.input_texts[idx],
            max_length=self.max_input_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        target_encoding = self.tokenizer(
            self.target_texts[idx],
            max_length=self.max_target_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": input_encoding["input_ids"].squeeze(),
            "attention_mask": input_encoding["attention_mask"].squeeze(),
            "labels": target_encoding["input_ids"].squeeze(),
        }



In [9]:
# Split the dataset into training and testing sets
train_input_texts, test_input_texts, train_target_texts, test_target_texts = train_test_split(
    input_texts, target_texts, test_size=0.1, random_state=42
)

# Create datasets for train, validation, and test
train_dataset = CustomDataset(tokenizer, train_input_texts, train_target_texts)
test_dataset = CustomDataset(tokenizer, test_input_texts, test_target_texts)

batch_size = 32
# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


## Model training

In [10]:
#set up training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [11]:
#training loop
loss = float('inf')

epochs = 65
model.train()
for epoch in range(epochs):
    epoch_loss = 0
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_dataloader)}")

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()
    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_dataloader)}")

100%|██████████| 2/2 [00:01<00:00,  1.04it/s]


Epoch 1/65, Loss: 6.983645915985107
Epoch 1/65, Validation Loss: 4.949277400970459


100%|██████████| 2/2 [00:00<00:00,  2.96it/s]


Epoch 2/65, Loss: 6.835392951965332
Epoch 2/65, Validation Loss: 4.703701972961426


100%|██████████| 2/2 [00:00<00:00,  2.96it/s]


...

Epoch 63/65, Loss: 1.6712150573730469
Epoch 63/65, Validation Loss: 2.355926036834717


100%|██████████| 2/2 [00:00<00:00,  2.95it/s]


Epoch 64/65, Loss: 1.6553191542625427
Epoch 64/65, Validation Loss: 2.3555562496185303


100%|██████████| 2/2 [00:00<00:00,  2.95it/s]

Epoch 65/65, Loss: 1.6172003746032715
Epoch 65/65, Validation Loss: 2.3557913303375244


## Evaluating of the Model

In [12]:

def evaluate_model(dataset, tokenizer, model):
    model.eval()
    predictions = []
    references = []
    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=256)
            predictions.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))
            references.extend([tokenizer.decode(ref, skip_special_tokens=True) for ref in batch["labels"]])
    return predictions, references

In [13]:
#run evaluation
predictions, references = evaluate_model(test_dataset, tokenizer, model)

Evaluating: 100%|██████████| 1/1 [00:03<00:00,  3.89s/it]


In [15]:

# Calculate ROUGE Scores
def calculate_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    rouge_scores = {"rouge1": [], "rouge2": [], "rougeL": []}
    for ref, pred in zip(references, predictions):
        scores = scorer.score(ref, pred)
        for key in rouge_scores:
            rouge_scores[key].append(scores[key].fmeasure)
    return {key: sum(values) / len(values) for key, values in rouge_scores.items()}

rouge_scores = calculate_rouge(predictions, references)
print(f"ROUGE Scores: {rouge_scores}")

ROUGE Scores: {'rouge1': 0.3840698292336229, 'rouge2': 0.13882919331745788, 'rougeL': 0.25124390540259856}


In [18]:
# Calculate BERT Scores
def calculate_bert_score(predictions, references):
    P, R, F1 = bert_score(predictions, references, model_type="bert-base-uncased", lang="en")
    return {
        "Precision": P.mean().item(),
        "Recall": R.mean().item(),
        "F1": F1.mean().item()
    }

bert_scores = calculate_bert_score(predictions, references)
print(f"BERT Scores: {bert_scores}")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERT Scores: {'Precision': 0.6510936617851257, 'Recall': 0.5887253284454346, 'F1': 0.6178234815597534}


In [19]:
# printing the summary predictions on the test set

# code to beautify the print statements
line_width = 145

for indx, val in enumerate(predictions):
    print(f"Reference {indx + 1}:\n{textwrap.fill(references[indx], width=line_width)}")
    print(f"Prediction {indx + 1}:\n{textwrap.fill(val, width=line_width)}")
    print("-" * 30)

Reference 1:
The Swiss Federal University for Vocational Education and Training (SFUVET) is based in Zollikofen and has 3 campuses in Switzerland. These
campuses are separated by the language spoken in the region, which are German, French, and Italilan. It colabortates with industry groups and
regional authorities to plan courses, services, and research. The government sets their goals and consults with national organikzations like
trade unions and employers associations. Old rules have been replaced on August 1, 2021, but a few parts will continue to be active until the end
of 2021.
Prediction 1:
The SFUVET Ordinance is established in Zollikofen, Switzerland. Its strategic objectives are to develop a partnership with professional
organizations and cantonal authorities in its strategic planning activities, training courses and research activities. It may also develop
advisory boards for professional organizations, cantonal authorities, and other interested parties. It repeals the 2005 

 references
  
 https://huggingface.co/docs/transformers/model_doc/t5

 model : https://huggingface.co/google/flan-t5-small

 tokenizer : https://huggingface.co/docs/transformers/main_classes/tokenizer

 train test split, sklearn : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

 dataloader : https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

 optimizer : https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html

 rogue score : https://huggingface.co/spaces/evaluate-metric/rouge/blob/main/README.md

 bert_score : https://github.com/Tiiiger/bert_score

 to beautify the print statements: https://www.geeksforgeeks.org/textwrap-text-wrapping-filling-python/